In [1]:
library(ReactomePA)
library(org.Sc.sgd.db)
library(clusterProfiler)
library(GOSim)
library(topGO)

Loading required package: DOSE

DOSE v3.0.10  For help: https://guangchuangyu.github.io/DOSE

If you use DOSE in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Guang-Rong Yan, Qing-Yu He. DOSE: an R/Bioconductor package for Disease Ontology Semantic and Enrichment analysis. Bioinformatics 2015, 31(4):608-609

ReactomePA v1.18.1  For help: https://guangchuangyu.github.io/ReactomePA

If you use ReactomePA in published research, please cite:
Guangchuang Yu, Qing-Yu He. ReactomePA: an R/Bioconductor package for reactome pathway analysis and visualization. Molecular BioSystems 2016, 12(2):477-479
Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, par

In [2]:
file <- "yeast_reactome"

ont <- "BP"
e_sg <- 0.7
e_en <- 0.6

db <- org.Sc.sgd.db
# mapping <- "org.Sc.sgd.db"
# ID <- "ENSEMBL"

##load all community gene lists
setwd(sprintf("/home/david/Documents/ghsom/hierarchical_exploration/%s_hierarchy_communities_%s_%s", file, e_sg, e_en))
# setwd(sprintf("/home/david/Desktop/%s_hierarchy_communities_%s", file, e_sg))

In [3]:
generateMap <- function(filename){
    map <- as.matrix(read.csv(filename, sep=",", header = F))
    communities <- map[,1]
    map <- map[,2:ncol(map)]
    rownames(map) <- communities
    colnames(map) <- communities
    return (map)
}

#background gene list
backgroundFilename <- "all_genes.txt"
allGenes <- scan(backgroundFilename, character())

##convert all genes to ENTREZID
conversion <- select(db, allGenes, "ENTREZID", "UNIPROT")
conversion <- subset(conversion, !duplicated(conversion$UNIPROT))
allGenes <- conversion$ENTREZID

#shortest path files
shortestPathFiles  <- list.files(pattern="*shortest_path*")

#shortest paths list
shortestPaths <- sapply(shortestPathFiles, generateMap)
names(shortestPaths) <- sapply(names(shortestPaths), function(name) strsplit(name, "_")[[1]][[1]])

#communitiy assignemtns
assignments <- as.matrix(read.csv("assignment_matrix.csv", sep=",", header=F, row.names=1, colClasses="character"))
assignments[assignments == ""] <- NA 
rownames(assignments) <- allGenes
colnames <- sapply(1:ncol(assignments), function(i) as.character(i - 1))
colnames(assignments) <- colnames
    
#filter out genes with no ENTREZID
assignments <- assignments[!is.na(rownames(assignments)),]
    
#all ORF identifers in org.Sc.sgd.db converted to EntrezID
allGenesInDB <- select(db, keys(db), "ENTREZID", "ORF")$ENTREZID
allGenesInDB <- allGenesInDB[!is.na(allGenesInDB)]
    
#communities detected
communities <- unique(as.character(assignments))
communities <- communities[communities != ""]
communities <- sort(communities)

'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


In [4]:
assignments

,0,1,2,3,4
851723,01,01-03,NA,NA,NA
853438,01,01-03,NA,NA,NA
856143,01,01-03,NA,NA,NA
851725,01,01-03,NA,NA,NA
856222,01,01-03,NA,NA,NA
856223,01,01-04,01-04-05,01-04-05-01,NA
851742,01,01-03,NA,NA,NA
851045,01,01-03,NA,NA,NA
854810,01,01-01,NA,NA,NA
851336,01,01-03,NA,NA,NA


In [5]:
getDepth <- function(com) {
    return(which(apply(assignments, 2, function(i) any(i == com))))
}

getGenes <- function(com){
    depth <- getDepth(com)
    return(names(which(assignments[, depth] == com)))
}

getSubCommunities <- function(com){
    depth <- getDepth(com)
    genesInCommunity <- subset(assignments, assignments[,depth] == com)
    if (depth < ncol(genesInCommunity)){
        return(as.character(unique(genesInCommunity[,depth + 1])))
    } else {
        return (NULL)
    }
    
}

getSuperCommunity <- function(com){
    depth <- getDepth(com)
    genesInCommunity <- subset(assignments, assignments[,depth] == com)
    return(as.character(unique(genesInCommunity[,depth - 1])))
}

getShortestPath <- function(com){
    return (try(shortestPaths[[com]]))
}
                       
getNeighbours <- function(com){
    
    superCommunity <- getSuperCommunity(com)
    superCommunityMap <- getShortestPath(superCommunity)
    v <- superCommunityMap[com, ] == 1
    return (names(v[v]))
    
}

In [6]:
genesInCommunities <- sapply(communities, function(i) getGenes(i))

In [7]:
genesInCommunities

$`01`
   [1] "851723" "853438" "856143" "851725" "856222" "856223" "851742" "851045"
   [9] "854810" "851336" "851336" "852254" "852254" "855892" "855423" "851358"
  [17] "852082" "852186" "854267" "854815" "854198" "851824" "852740" "850673"
  [25] "850642" "851413" "850343" "855118" "850674" "854231" "851488" "855587"
  [33] "851985" "854965" "856626" "856403" "856499" "856389" "856388" "855264"
  [41] "856387" "854276" "852180" "851447" "850711" "855726" "851501" "852431"
  [49] "851058" "853674" "852376" "854353" "850592" "855836" "850750" "856312"
  [57] "854297" "851461" "856317" "852451" "856547" "854399" "852406" "855636"
  [65] "853301" "856478" "853587" "856731" "854338" "852071" "852762" "855977"
  [73] "853252" "851122" "851814" "855981" "854717" "853552" "851856" "856476"
  [81] "852890" "854263" "855230" "856777" "856763" "851558" "854918" "852973"
  [89] "855228" "851404" "851620" "855155" "852344" "852385" "856024" "850543"
  [97] "853566" "855602" "852696" "853574" "851798" "853573" "856692" "854472"
 [105] "850877" "854468" "852673" "856924" "856909" "850998" "851797" "854138"
 [113] "852955" "855625" "855948" "855737" "853287" "851493" "855459" "853585"
 [121] "856804" "852793" "853643" "853642" "855441" "855036" "851671" "853174"
 [129] "851670" "855675" "851068" "851421" "852308" "851193" "853842" "854221"
 [137] "854256" "853798" "855844" "856411" "854527" "853733" "850713" "855631"
 [145] "850397" "850839" "850732" "855950" "852306" "850973" "850882" "851035"
 [153] "856398" "855082" "852101" "852394" "856322" "855741" "856394" "854881"
 [161] "850322" "855739" "850454" "853859" "853632" "855460" "852230" "851480"
 [169] "856415" "852003" "854811" "856462" "854517" "851666" "856574" "852467"
 [177] "854382" "853096" "854791" "853051" "856801" "855447" "851960" "856311"
 [185] "850437" "852530" "851364" "850977" "851860" "854165" "855621" "856217"
 [193] "853062" "852317" "856382" "854776" "856648" "854678" "850966" "855346"
 [201] "853904" "850712" "856154" "855319" "853242" "855221" "854290" "854301"
 [209] "852211" "851141" "852440" "854584" "852185" "852489" "852330" "850298"
 [217] "855936" "853393" "853600" "850430" "851159" "856101" "854243" "856438"
 [225] "856593" "851733" "854187" "855655" "851830" "852700" "851833" "850317"
 [233] "853109" "856252" "854332" "856687" "855283" "855560" "853557" "856694"
 [241] "851499" "854551" "852220" "854663" "851076" "856250" "855560" "854942"
 [249] "855929" "851713" "853639" "855454" "855267" "856087" "851203" "855083"
 [257] "852838" "853644" "855302" "855725" "853869" "855101" "855514" "853118"
 [265] "852617" "850793" "856831" "854749" "854673" "852833" "852060" "851019"
 [273] "853896" "852775" "854984" "851990" "854946" "854179" "855609" "855593"
 [281] "855688" "856693" "855571" "855232" "855705" "854646" "854505" "850613"
 [289] "855357" "852665" "853056" "854119" "855362" "852607" "856891" "853557"
 [297] "856607" "853734" "851332" "855170" "850930" "851676" "853902" "856691"
 [305] "854875" "853147" "850589" "850314" "850514" "853520" "855245" "853155"
 [313] "852224" "853003" "856310" "856291" "852108" "852421" "856255" "850422"
 [321] "853997" "853783" "855968" "855078" "856899" "854786" "856294" "856835"
 [329] "856823" "851386" "855626" "854229" "851520" "852853" "853807" "850554"
 [337] "851906" "850620" "855701" "850764" "854241" "850905" "850358" "851485"
 [345] "855203" "854387" "856859" "851641" "853286" "855732" "856375" "856422"
 [353] "853536" "856553" "850919" "852543" "855287" "852207" "852577" "852240"
 [361] "854342" "850611" "855832" "851543" "855543" "850790" "852487" "853668"
 [369] "851636" "850864" "853167" "856523" "852083" "855104" "856119" "854385"
 [377] "854976" "856085" "852614" "856237" "855778" "855429" "851874" "851158"
 [385] "856395" "852675" "855983" "855237" "856201" "853161" "851584" "852888"
 [393] "853002" "856236" "853421" "851115" "853243" "852288" "852206" "852037"
 [401] "854346" "853304" "853307" "851878" "

In [8]:
lengths(genesInCommunities)

01          01-01          01-02       01-02-01    01-02-01-01 
          1131            130             69             26              7 
   01-02-01-02    01-02-01-03    01-02-01-04       01-02-02    01-02-02-01 
             7              9              3             23              7 
   01-02-02-02    01-02-02-03       01-02-03          01-03          01-04 
            10              6             20            702            230 
      01-04-01    01-04-01-01    01-04-01-02    01-04-01-03       01-04-02 
            21             13              6              2             37 
   01-04-02-01    01-04-02-02    01-04-02-03       01-04-03       01-04-04 
            19              7             11            101             20 
   01-04-04-01    01-04-04-02    01-04-04-03    01-04-04-04       01-04-05 
             6              8              3              3             29 
   01-04-05-01    01-04-05-02    01-04-05-03 01-04-05-03-01 01-04-05-03-02 
             7             10             12              2              3 
01-04-05-03-03       01-04-06       01-04-07    01-04-07-01    01-04-07-02 
             7             11             11              4              3 
   01-04-07-03 
             4

In [9]:
enrichmentResults <- sapply(genesInCommunities, 
                            function (i) enrichPathway(gene = i, universe = allGenesInDB, organism = "yeast"))
names(enrichmentResults) <- communities

No gene can be mapped....
--> return NULL...
No gene can be mapped....
--> return NULL...


In [10]:
sapply(enrichmentResults, function(i) nrow(as.data.frame(i)))

01          01-01          01-02       01-02-01    01-02-01-01 
            88              0              0              0              0 
   01-02-01-02    01-02-01-03    01-02-01-04       01-02-02    01-02-02-01 
             0              0              0              0              0 
   01-02-02-02    01-02-02-03       01-02-03          01-03          01-04 
             0              3              0              0             12 
      01-04-01    01-04-01-01    01-04-01-02    01-04-01-03       01-04-02 
             0              0              0             26              0 
   01-04-02-01    01-04-02-02    01-04-02-03       01-04-03       01-04-04 
             0              0              0              0              0 
   01-04-04-01    01-04-04-02    01-04-04-03    01-04-04-04       01-04-05 
             0              0              0              0              0 
   01-04-05-01    01-04-05-02    01-04-05-03 01-04-05-03-01 01-04-05-03-02 
             0              0              0              0              0 
01-04-05-03-03       01-04-06       01-04-07    01-04-07-01    01-04-07-02 
             0              0              0              0              0 
   01-04-07-03 
             0

In [11]:
x  <- enrichmentResults[["01-02"]]
head(as.data.frame(x))

ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [12]:
# x <- enrichmentResults[["01-05"]]

In [13]:
# nrow(as.data.frame(x))

In [39]:
# barplot(x, showCategory=10, title = "Top Enriched Pathways")

In [40]:
# dotplot(x, showCategory=15)

In [41]:
# enrichMap(x, layout=igraph::layout.kamada.kawai, vertex.label.cex = 1)

In [29]:
numbersOfEnrichedPathways <- sapply(enrichmentResults, function(i) nrow(as.data.frame(i)))
enrichedCommunities <- genesInCommunities[numbersOfEnrichedPathways > 0]

In [30]:
res <- compareCluster(enrichedCommunities, 
                      fun="enrichPathway", universe = allGenesInDB, organism = "yeast")

png(filename=sprintf("community_pathway_enrichment_all_communities.png"), width=1500)
plot(res)
dev.off()

png 
  2

In [31]:
plotPathwayEnrichments <- function(community){
    
    subCommunities <- getSubCommunities(community)
    
    if (!is.null(subCommunities) && !any(is.na(subCommunities) > 0)) {

        communitiesOfInterest <- c(community, subCommunities)
#         print(communitiesOfInterest)
        genesOfInterest <- enrichedCommunities[communitiesOfInterest]
        genesOfInterest <- genesOfInterest[!is.na(names(genesOfInterest))]
#         print (genesOfInterest)
        
        if (length(genesOfInterest) > 2) {
            res <- compareCluster(genesOfInterest,
            fun="enrichPathway", universe = allGenesInDB, organism = "yeast")

            png(filename=sprintf("community_pathway_enrichment_%s.png", community), width=500 + length(genesOfInterest) * 150)
            print(plot(res))
            dev.off()
        } 
        
    }

}

In [32]:
sapply(communities, plotPathwayEnrichments)

$`01`
png 
  2 

$`01-01`
NULL

$`01-02`
NULL

$`01-03`
NULL

$`01-04`
NULL

$`01-04-01`
NULL

$`01-04-02`
NULL

$`01-04-03`
NULL

$`01-04-04`
NULL

$`01-05`
png 
  2 

$`01-05-01`
NULL

$`01-05-02`
NULL

$`01-05-03`
NULL

$`01-05-04`
NULL

$`01-05-05`
NULL

$`01-05-06`
NULL


In [42]:
# viewPathway(pathName = "Nonsense Mediated Decay (NMD) enhanced by the Exon Junction Complex (EJC)", 
#             organism = "yeast", readable = F)

In [9]:
setOntology(ont, loadIC=TRUE)
setEvidenceLevel(evidences="all", organism=org.Sc.sgdORGANISM, gomap=org.Sc.sgdGO)

initializing GOSim package ...
-> retrieving GO information for all available genes for organism 'human' in GO database
-> filtering GO terms according to evidence levels 'all'
-> loading files with information content for corresponding GO category (human)
finished.
-> loading files with information content for corresponding GO category (human)
-> retrieving GO information for all available genes for organism 'Saccharomyces cerevisiae' in GO database
-> filtering GO terms according to evidence levels 'all'


In [12]:
allGenesORF <- keys(db)


GOenrichmentResults <- sapply(genesInCommunities, function(genesOfInterest) {
    
    conversionTable <- select(db, genesOfInterest, "ORF", "ENTREZID")
    
    GOenrichment(conversionTable$ORF, allGenesORF, cutoff=0.05, method="weight01")
}
)

'select()' returned many:1 mapping between keys and columns

Building most specific GOs .....
	( 2909 GO terms found. )

Build GO DAG topology ..........
	( 5064 GO terms and 11404 relations. )

Annotating nodes ...............
	( 6419 genes annotated to the GO terms. )

			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 3195 nontrivial nodes
		 parameters: 
			 test statistic: fisher

	 Level 17:	1 nodes to be scored	(0 eliminated genes)

	 Level 16:	12 nodes to be scored	(0 eliminated genes)

	 Level 15:	36 nodes to be scored	(2 eliminated genes)

	 Level 14:	75 nodes to be scored	(40 eliminated genes)

	 Level 13:	138 nodes to be scored	(213 eliminated genes)

	 Level 12:	186 nodes to be scored	(516 eliminated genes)

	 Level 11:	281 nodes to be scored	(1014 eliminated genes)

	 Level 10:	380 nodes to be scored	(1600 eliminated genes)

	 Level 9:	447 nodes to be scored	(2277 eliminated genes)

	 Level 8:	405 nodes to be scored	(3043 eliminated genes)

	 Level 7:	413 nodes to

In [15]:
rownames(GOenrichmentResults) <- c("terms", "p-values", "genes")